In [1]:
import os

import numpy as np
from tqdm import tqdm
from scipy.stats import truncnorm
import sys
import warnings

sys.path.append('../')
sys.path.append('../../')
from DataBot.data_bot import BotLGBM
from Boards.Speed_leedo_2p_v6 import FullBoard
from Intelligent_bot.intelligent_bot import Bot
import gc

env = FullBoard()
agent0 = BotLGBM()
bot0 = Bot()

wins = []

agent0.load_model('../../DataBot/models/lgbm_bot_v2.pkl')

warnings.filterwarnings('ignore')

for ep in tqdm(range(0, 10000), ascii=True, unit="e"):
    s, _, game_over, player_turn = env.reset()
    episode_reward = [0.0, 0.0]

    time = 0
    while time < 360 or player_turn == 0:

        time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
        player_turn_temp = env.get_player_turn()
        env.roll_dice()[0]
        player_turn = env.get_player_turn()

        if player_turn == player_turn_temp:

            action_list = env.get_next_states(player_turn)

            if action_list:
                if player_turn == 0:
                    s_t = env.return_state()
                    action = agent0.act(s_t, action_list)
                else:
                    s_t = env.return_state()
                    action = bot0.act(state=s_t, p=player_turn)

                time += truncnorm.rvs(-2.5, 2.5, loc=2.5, scale=1)
                s_, reward, game_over, player_turn_temp, game_reward  = env.make_step(action)

                episode_reward[player_turn] += reward[player_turn]
       
    if game_reward[0] > game_reward[1]:
        wins.append(1)
    else:
        wins.append(0)

    if ep % 1000 == 0 and ep != 0:
        print(np.sum(wins))

2022-10-26 11:18:58.796725: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 11:18:58.811050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 11:18:58.811885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
 10%|#         | 1003/10000 [01:27<11:19, 13.24e/s]

546


 20%|##        | 2001/10000 [02:54<10:41, 12.46e/s]

1092


 30%|###       | 3002/10000 [04:23<11:03, 10.55e/s]

...

 80%|########  | 8002/10000 [11:57<02:24, 13.80e/s]

4363


 90%|######### | 9002/10000 [13:32<01:33, 10.69e/s]

4902


100%|##########| 10000/10000 [15:06<00:00, 11.03e/s]


In [2]:
np.sum(wins)

5428